In [168]:
import pandas as pd
import statsmodels.formula.api as sm
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [177]:
data = pd.read_csv('./datasets/data.csv', encoding='CP949')
target = pd.read_csv('./datasets/OECD신용등급.csv', encoding='CP949')


In [178]:
data

,국가별,시점,국내총생산,고용률,국민부담률,경제활동참가율 (%),무역의존도_수출,무역의존도_수입,수출,수입,외환보유액,부채비율,실업률,평균근로자세금,디플레이터,GDP_성장률
0,한국,2000,12261,61.5,20.9,61.2,29.89,27.84,172268.0,160481.0,96198.0,NaN,4.4,16.4,72.1,9.1
1,한국,2005,19399,63.7,21.7,62.2,30.43,27.95,284419.0,261238.0,210391.0,NaN,3.7,17.3,82.9,4.3
2,한국,2010,23083,63.4,22.4,61.1,40.77,37.17,466384.0,425212.0,291571.0,NaN,3.7,20.1,92.7,6.8
3,한국,2015,28724,65.9,23.7,62.8,35.95,29.79,526757.0,436499.0,367962.0,40.8,3.6,21.4,100.0,2.8
4,한국,2020,31637,65.9,NaN,62.5,31.28,28.55,512498.0,467633.0,443098.0,NaN,4.0,23.3,105.2,-0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,뉴질랜드,2000,13641,70.3,32.5,65.2,25.27,26.42,13298.0,13906.0,3952.0,NaN,6.1,19.4,77.0,2.9
189,뉴질랜드,2005,27751,74.2,36.1,67.7,18.94,22.87,21728.0,26233.0,8893.0,NaN,3.8,20.0,86.8,3.3
190,뉴질랜드,2010,33677,72.1,30.3,67.6,21.36,20.91,31301.0,30635.0,16723.0,NaN,6.6,17.0,100.0,1.5
191,뉴질랜드,2015,38631,74.2,31.6,68.8,19.29,20.53,34354.0,36550.0,14700.0,NaN,5.4,17.6,108.5,3.7


In [179]:
datasets = pd.merge(data, target, on=('국가별', '시점'), how='left')
datasets.columns = ['국가별', '시점', '국내총생산', '고용률', '국민부담률', '경제활동참가율', '무역의존도_수출',
       '무역의존도_수입', '수출', '수입', '외환보유액', '부채비율', '실업률', '평균근로자세금', '디플레이터',
       'GDP_성장률', '신용등급']
datasets.dropna(inplace=True)
datasets.reset_index(inplace=True)
datasets.drop(['국가별', 'index'], axis=1, inplace=True)
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시점        116 non-null    int64  
 1   국내총생산     116 non-null    int64  
 2   고용률       116 non-null    float64
 3   국민부담률     116 non-null    float64
 4   경제활동참가율   116 non-null    float64
 5   무역의존도_수출  116 non-null    float64
 6   무역의존도_수입  116 non-null    float64
 7   수출        116 non-null    float64
 8   수입        116 non-null    float64
 9   외환보유액     116 non-null    float64
 10  부채비율      116 non-null    float64
 11  실업률       116 non-null    float64
 12  평균근로자세금   116 non-null    float64
 13  디플레이터     116 non-null    float64
 14  GDP_성장률   116 non-null    float64
 15  신용등급      116 non-null    float64
dtypes: float64(14), int64(2)
memory usage: 14.6 KB


In [180]:
lb = LabelEncoder()
datasets['시점'] = lb.fit_transform(datasets['시점'])
datasets.describe()

,시점,국내총생산,고용률,국민부담률,경제활동참가율,무역의존도_수출,무역의존도_수입,수출,수입,외환보유액,부채비율,실업률,평균근로자세금,디플레이터,GDP_성장률,신용등급
count,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,1.160000e+02,1.160000e+02,1.160000e+02,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,1.689655,33346.146552,65.972414,34.313793,60.153448,37.006552,37.020345,2.217822e+05,2.229416e+05,8.167653e+04,73.539026,8.257759,38.507759,95.595690,3.329310,16.979940
std,1.058414,20702.131682,7.047338,6.950307,5.940245,22.166097,21.632006,2.805340e+05,3.268235e+05,1.823440e+05,41.213343,4.170646,8.705592,13.460676,3.148085,3.344374
min,0.000000,4076.000000,46.300000,11.400000,48.100000,7.440000,2.240000,3.092000e+03,3.919000e+03,2.790000e+02,6.788000,2.600000,14.700000,58.200000,-5.500000,3.900000
25%,1.000000,16641.000000,60.650000,30.500000,57.025000,20.292500,22.902500,4.842350e+04,4.620275e+04,9.268750e+03,45.829500,5.100000,32.825000,87.700000,1.800000,14.500000
50%,2.000000,30719.000000,66.100000,33.100000,59.650000,31.785000,29.410000,1.204630e+05,8.910150e+04,3.073500e+04,63.252000,7.450000,38.800000,100.000000,2.950000,18.000000
75%,3.000000,44260.500000,71.575000,39.250000,63.125000,51.107500,50.787500,3.073575e+05,3.133178e+05,7.170775e+04,101.067500,9.925000,43.775000,103.700000,4.200000,20.000000
max,3.000000,104965.000000,83.800000,48.000000,81.400000,146.180000,137.700000,1.503400e+06,2.248800e+06,1.233153e+06,234.073000,24.900000,57.100000,143.100000,25.200000,20.000000


In [181]:
np.random.seed(42)

len_ = len(datasets)
a = np.random.permutation(np.arange(len_))

index_ = int(len_*0.8)
train_index = a[:index_]
test_index = a[index_:]

In [182]:
train_df = datasets.loc[train_index]
test_df = datasets.loc[test_index]

In [183]:
ols_ = sm.ols('신용등급 ~ 시점 + 국내총생산 + 고용률 + 국민부담률 + 경제활동참가율 +  무역의존도_수출 + 무역의존도_수입 + 수출 + 수입 + 외환보유액 + 부채비율 + 실업률 + 평균근로자세금 + 디플레이터 + GDP_성장률',
 data=train_df)
result = ols_.fit()
print(result.summary())


                            OLS Regression Results                            
Dep. Variable:                   신용등급   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                     19.34
Date:                Wed, 11 May 2022   Prob (F-statistic):           4.75e-20
Time:                        19:30:47   Log-Likelihood:                -166.04
No. Observations:                  92   AIC:                             364.1
Df Residuals:                      76   BIC:                             404.4
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.6984      4.111      0.900      0.3